In [15]:
import pandas as pd

# Load the data
data = pd.read_csv('demmo_data.csv')
data.head()

,Emotional_Word,RGB_1,RGB_2,RGB_3
0,Cute,"[251, 167, 157]","[255, 242, 124]","[179, 22, 61]"
1,Childlike,"[251, 103, 89]","[255, 242, 124]","[153, 216, 212]"
2,Pretty,"[251, 167, 157]","[255, 242, 63]","[78, 181, 135]"
3,Sweet,"[251, 103, 89]","[253, 192, 145]","[251, 174, 193]"
4,Amusing,"[253, 166, 74]","[140, 201, 25]","[90, 177, 132]"


Preprocess the data 

In [22]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv('demmo_data.csv')

# Function to convert string representation of lists into actual lists
def string_to_array(s):
    return np.array(eval(s))

# Apply the function to each RGB column
data['RGB_1'] = data['RGB_1'].apply(string_to_array)
data['RGB_2'] = data['RGB_2'].apply(string_to_array)
data['RGB_3'] = data['RGB_3'].apply(string_to_array)

# Combine RGB columns into a single feature matrix
X = np.hstack((np.vstack(data['RGB_1']), np.vstack(data['RGB_2']), np.vstack(data['RGB_3'])))

# Create labels (assuming one-hot encoding is needed)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data['Emotional_Word'])

onehot_encoder = OneHotEncoder(sparse_output=False)
y = onehot_encoder.fit_transform(integer_encoded.reshape(-1, 1))

# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the data
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (182, 9)
y_train shape: (182, 155)
X_test shape: (46, 9)
y_test shape: (46, 155)


Build the MLP model

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define a simpler model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train the model

In [24]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=16, callbacks=[early_stopping])


Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.0029 - loss: 79.9304 - val_accuracy: 0.0217 - val_loss: 79.9351
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0106 - loss: 76.8806 - val_accuracy: 0.0217 - val_loss: 74.6050
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0018 - loss: 70.7986 - val_accuracy: 0.0217 - val_loss: 69.7658
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0029 - loss: 65.5066 - val_accuracy: 0.0217 - val_loss: 65.2088
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0047 - loss: 59.3625 - val_accuracy: 0.0000e+00 - val_loss: 61.1458
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0118 - loss: 54.1501 - val_accuracy: 0.0000e+00 - val_loss: 57.5344
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0166 - loss: 51.6542 - val_accuracy: 0.0000e+00 - val_loss: 54.3490
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0021 - loss: 53.08

Evaluate the model

In [25]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 7.0828  
Test loss: 6.947267532348633
Test accuracy: 0.0


Hyperparameter Tuning

In [20]:
from keras_tuner import RandomSearch

# Define a function to build the model (for hyperparameter tuning)
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=512, step=32), input_dim=X_train.shape[1], activation='relu'))
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

# Create a tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='hyperparam_tuning', project_name='mlp_emotion')

# Perform the hyperparameter search
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
96                |96                |units_input
1                 |1                 |num_layers
288               |288               |units_0
0.0043741         |0.0043741         |learning_rate



FatalValueError: All callbacks used during a search should be deep-copyable (since they are reused across trials). It is not possible to do `copy.deepcopy([<keras.src.callbacks.early_stopping.EarlyStopping object at 0x000002193D2A4470>])`

Get the best hyperparameters and retrain the model

In [ ]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
best_loss, best_accuracy = best_model.evaluate(X_test, y_test)
print(f'Best test loss: {best_loss}')
print(f'Best test accuracy: {best_accuracy}')
